In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn; seaborn.set()

# задание фрейма "в лоб": x = pd.DataFrame({'столбец1': значение,
#                                           'столбец2': значение}, index=значение)
# значение - переменная, число, список (в виде [a,b,c])
# также, в качестве data можно передавать зарнее заготовленный словарь


# для pd.Series можно задать имя, которое станет именнем столбца данной Series
# при её импорте в DataFrame (при условии что при содании фрейма мы НЕ задали
# имя столбца, а просто импортировали Series: pd.DataFrame(наша_Series).
# Имя Series задается как name='имя'

area = pd.Series({'Cal': 423967,
                'Tex': 695662,
                'New': 141297,
                'Flo': 170312,
                'Ill': 149995}, name='kek') # это имя видно не будет, т.к. при
                               # создании фрейма мы сами задали имена столбцов

pop = pd.Series({'Cal': 38332521,
                'Tex': 26448193,
                'New': 19651127,
                'Flo': 19552860,
                'Ill': 12882135})

data = pd.DataFrame({'area': area,
                    'pop': pop})

data['density'] = data['pop']/data['area']
data.iloc[0,2] = 90
data[data.density > 100]

,area,pop,density
New,141297,19651127,139.076746
Flo,170312,19552860,114.806121


In [2]:
data = pd.Series([1, np.nan, 'hello', None]) # NaN (из numpy) и None (из питона) рассматриваются пандасом как одно и то же
df = pd.DataFrame([[1, np.nan, 2],
                  [2, 3, 5],
                  [np.nan, 4, 6]])

data[data.isnull()] # вывод нулевых/ненулевых значений Series/DataFrame
data.dropna # для series удаляются все нулевые элементы
# а вот в DataFrame удалить отдельные элементы нельзя. Поэтому удаляются либо строки, либо столбцы

df.dropna(axis='rows') # чтоб было удобнее читать код пишем не axis=1/0, а axis='colums'/'rows'

df.dropna(axis='rows', how='all') # how='all' - параметр отбрасывания пустых строк/столбцов
# при 'all' - удаляются только строки/столбцы в которых ВСЕ элементы нулевые
# при 'any' - удаляются строки/столбцы в которых хотя бы один элемент нулевой

df.dropna(axis='rows', thresh=2) # thresh - минимальное количество НЕНУЛЕВЫХ значений при котором
# строка/столбец НЕ отбрасывается

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [3]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data.fillna(0) # заполнение всех нулевых значений каким-то ненулевым значением
# у pd.fillna() есть много методов заполнения - читай документацию к методу

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [4]:
# "сервисные" функции для упрощения работы с темой
# выходят за рамки книги, однако полезны

def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

# используется для удобного отображения нескольких датафреймов рядом с друг другом
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)


In [7]:
# CONCATENATION
# pd.concat - метод объединения датафреймов и сириесов

df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2], axis=0, verify_integrity=True)')
#verify_integrity=True - проверка на повторение индексов, если они повторяются вылезет ошибка

x = make_df('AB', [10, 20])
y = make_df('AB', [30, 40])
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')
# ignore_index=True - удаление имеющихся индексов и задание в новом датафрейме новых индексов

display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")
# keys - задание мультииндексов для нового датафрейма
# датафреймы из которых он был составлен получат новый индекс поверх своих старых


x
      A    B
10  A10  B10
20  A20  B20

y
      A    B
30  A30  B30
40  A40  B40

pd.concat([x, y], keys=['x', 'y'])
        A    B
x 10  A10  B10
  20  A20  B20
y 30  A30  B30
  40  A40  B40

In [6]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6], join_axes=[df5.columns])')
# join_axes - по каким осям проводить объединение (читай документацию)

,A,B,C
1,A1,B1,C1
2,A2,B2,C2
,B,C,D
3,B3,C3,D3
4,B4,C4,D4
,A,B,C
1,A1,B1,C1
2,A2,B2,C2
3,NaN,B3,C3
4,NaN,B4,C4
